In [1]:
# Setup the initial variables
run_start = time.time()
data_dir = os.path.join("..","cs-train")
print("...fetching data")

# Fetching data may take a few seconds
df = fetch_data(data_dir)

# Calculate time taken and display
m, s = divmod(time.time()-run_start,60)
h, m = divmod(m, 60)
print("load time:", "%d:%02d:%02d"%(h, m, s))

# Print the data
df.head()

# Convert the dataframe to a timeseries
ts = convert_to_ts(df)
ts.head()

# Create new features for training
X, y, dates = engineer_features(ts)

# The feature data
X.head()

Hello World!


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, median_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

# Tách tập train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

# Tạo các pipeline khác nhau cho từng mô hình
models = {
    "Random Forest": Pipeline([
        ("scaler", StandardScaler()),
        ("model", RandomForestRegressor(n_estimators=100, random_state=0))
    ]),
    "AdaBoost": Pipeline([
        ("scaler", StandardScaler()),
        ("model", AdaBoostRegressor(n_estimators=100, learning_rate=0.8, random_state=0))
    ]),
    "XGBoost": Pipeline([
        ("scaler", StandardScaler()),
        ("model", XGBRegressor(
            n_estimators=200,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=0,
            objective='reg:squarederror'
        ))
    ])
}

# Huấn luyện và đánh giá từng mô hình
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = median_absolute_error(y_test, y_pred)
    print(f"{name:15s} → R² = {r2:.3f}, MAE = {mae:.3f}")
